### 1. 

En el archivo "logistic_regression_df_class" hemos visto un ejemplo multiclase. Elimina del dataframe todas las filas que se correspondan con la clase valor "1".

Ahora, realiza el ejercicio con el nuevo dataframe:

- ¿Se mejora la precisión del algoritmo con dos clases? ¿por qué?

LogisticRegression() es una clase que tiene varios parámetros de entrada. Investiga (modifica, prueba) los argumentos y comenta si modificando algunas de ellas se mejora el porcentaje de acierto del problema (probar al menos 2 diferentes)

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html


In [287]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
import random
import seaborn as sns

In [288]:
dataframe = pd.read_csv("../data/usuarios_win_mac_lin.csv")
dataframe

,duracion,paginas,acciones,valor,clase
0,7.0,2,4,8,2
1,21.0,2,6,6,2
2,57.0,2,4,4,2
3,101.0,3,6,12,2
4,109.0,2,6,12,2
...,...,...,...,...,...
165,13.0,1,7,42,0
166,12.0,1,2,6,0
167,13.0,1,3,9,0
168,13.0,1,7,28,0


In [289]:
dataframe.clase.value_counts()

0    86
2    44
1    40
Name: clase, dtype: int64

In [290]:
dataframe.drop(dataframe[dataframe.clase == 1].index, inplace = True)

In [291]:
dataframe

,duracion,paginas,acciones,valor,clase
0,7.0,2,4,8,2
1,21.0,2,6,6,2
2,57.0,2,4,4,2
3,101.0,3,6,12,2
4,109.0,2,6,12,2
...,...,...,...,...,...
165,13.0,1,7,42,0
166,12.0,1,2,6,0
167,13.0,1,3,9,0
168,13.0,1,7,28,0


In [292]:
dataframe.clase.value_counts()

0    86
2    44
Name: clase, dtype: int64

In [293]:
X = np.array(dataframe.drop(['clase'], axis=1))
y = np.array(dataframe['clase'])
print('y:', y.shape)
print('X:', X.shape)

y: (130,)
X: (130, 4)


## Validación del modelo

In [294]:
seed = 7
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

In [295]:
model = LogisticRegression(max_iter=100)
name='Logistic Regression'
kfold = KFold(n_splits=10, random_state = seed, shuffle = True) #Parte los datos en 10 trozos para usar validación cruzada / cross validation
cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy',)
msg = "%s: %f %s (%f)" % (name, cv_results.mean(), "+-", cv_results.std())
print(cv_results)
print("----------")
print(msg)
print("----------")
print("----------")

[1.         0.90909091 0.90909091 0.90909091 1.         0.8
 1.         1.         0.9        0.8       ]
----------
Logistic Regression: 0.922727 +- (0.074218)
----------
----------


In [296]:
# Los porcentajes obtenidos en el Cross Validation me satisfacen, entonces, entreno los datos del conjunto TRAIN

model.fit(X_train, Y_train)

LogisticRegression()

In [297]:
# Veamos qué porcentaje de acierto tengo para mi conjunto TRAIN

model.score(X_train, Y_train)

0.9519230769230769

In [298]:
# Veamos qué porcentaje de acierto tengo para mi conjunto TEST (éstos son los datos que yo no le di, y es importante que el valor sea al menos cercano a los obtenidos para CV y para el conjunto TRAIN)

model.score(X_test, Y_test)

0.9615384615384616

Se verifica un buen porcentaje para el conjunto "TEST"

In [299]:
# Para el conjunto TEST, el primer valor es el siguiente
Y_test[0::]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 2, 0], dtype=int64)

In [300]:
model.predict(X_test)[8]

0

In [301]:
# Veamos si la predicción  es correcta
model.predict(X_test[0:1])

array([0], dtype=int64)

In [302]:
# La predicción fue correcta... pero, qué tan seguro estaba de esta predicción ??
model.predict_proba(X_test[0:1])

array([[1.00000000e+00, 5.34724688e-18]])

Ésto último nos dice que estuvo prácticamete un 100% seguro de que se trataba de una clase 0 y no 1.

## Pruebas cambiando algunas variables:

In [307]:
seed = 1
ts = 0.15
mi = 200
ns = 10
print('Seed:', seed)
print('Test size:', ts)
print('Max iter:', mi)
print('Número splits:', ns)

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=ts, random_state=seed)
model = LogisticRegression(max_iter=mi)
name='Logistic Regression'
kfold = KFold(n_splits=ns, random_state = seed, shuffle = True) #Parte los datos en 10 trozos para usar validación cruzada / cross validation
cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy',)
msg = "%s: %f %s (%f)" % (name, cv_results.mean(), "+-", cv_results.std())
print("----------")
print('Resultados de CV:', cv_results)
print("----------")
print(msg)
print("----------")
model.fit(X_train, Y_train)
print('Score TRAIN:', model.score(X_train, Y_train))
print('Score TEST:', model.score(X_test, Y_test))
print("----------")
Y_test_random = random.randint(0,len(Y_test)-1)
print('Valor de Y_test en ' + str(Y_test_random) + ':', Y_test[Y_test_random])
print('Predicción de Y_test:', model.predict(X_test)[Y_test_random])
print('Seguridad en predicción de Y_test:', model.predict_proba(X_test)[Y_test_random])

Seed: 1
Test size: 0.2
Max iter: 200
Número splits: 10
----------
Resultados de CV: [1.         1.         0.81818182 1.         1.         0.9
 0.9        1.         1.         0.8       ]
----------
Logistic Regression: 0.941818 +- (0.076946)
----------
Score TRAIN: 0.9615384615384616
Score TEST: 0.8846153846153846
----------
Valor de Y_test en 17: 0
Predicción de Y_test: 0
Seguridad en predicción de Y_test: [9.99999980e-01 2.03398613e-08]


In [306]:
seed = 2
ts = 0.15
mi = 200
ns = 10
print('Seed:', seed)
print('Test size:', ts)
print('Max iter:', mi)
print('Número splits:', ns)

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=ts, random_state=seed)
model = LogisticRegression(max_iter=mi)
name='Logistic Regression'
kfold = KFold(n_splits=ns, random_state = seed, shuffle = True) #Parte los datos en 10 trozos para usar validación cruzada / cross validation
cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy',)
msg = "%s: %f %s (%f)" % (name, cv_results.mean(), "+-", cv_results.std())
print("----------")
print('Resultados de CV:', cv_results)
print("----------")
print(msg)
print("----------")
model.fit(X_train, Y_train)
print('Score TRAIN:', model.score(X_train, Y_train))
print('Score TEST:', model.score(X_test, Y_test))
print("----------")
Y_test_random = random.randint(0,len(Y_test)-1)
print('Valor de Y_test en ' + str(Y_test_random) + ':', Y_test[Y_test_random])
print('Predicción de Y_test:', model.predict(X_test)[Y_test_random])
print('Seguridad en predicción de Y_test:', model.predict_proba(X_test)[Y_test_random])

Seed: 2
Test size: 0.15
Max iter: 200
Número splits: 10
----------
Resultados de CV: [1.         0.90909091 1.         1.         0.90909091 1.
 0.72727273 0.90909091 0.90909091 1.        ]
----------
Logistic Regression: 0.936364 +- (0.081818)
----------
Score TRAIN: 0.9545454545454546
Score TEST: 0.95
----------
Valor de Y_test en 0: 0
Predicción de Y_test: 0
Seguridad en predicción de Y_test: [9.99999675e-01 3.25224113e-07]


In [310]:
seed = 2
ts = 0.15
mi = 200
ns = 30
print('Seed:', seed)
print('Test size:', ts)
print('Max iter:', mi)
print('Número splits:', ns)

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=ts, random_state=seed)
model = LogisticRegression(max_iter=mi)
name='Logistic Regression'
kfold = KFold(n_splits=ns, random_state = seed, shuffle = True) #Parte los datos en 10 trozos para usar validación cruzada / cross validation
cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy',)
msg = "%s: %f %s (%f)" % (name, cv_results.mean(), "+-", cv_results.std())
print("----------")
print('Resultados de CV:', cv_results)
print("----------")
print(msg)
print("----------")
model.fit(X_train, Y_train)
print('Score TRAIN:', model.score(X_train, Y_train))
print('Score TEST:', model.score(X_test, Y_test))
print("----------")
Y_test_random = random.randint(0,len(Y_test)-1)
print('Valor de Y_test en ' + str(Y_test_random) + ':', Y_test[Y_test_random])
print('Predicción de Y_test:', model.predict(X_test)[Y_test_random])
print('Seguridad en predicción de Y_test:', model.predict_proba(X_test)[Y_test_random])

Seed: 2
Test size: 0.15
Max iter: 200
Número splits: 30
----------
Resultados de CV: [1.         1.         1.         0.75       1.         1.
 1.         1.         1.         1.         1.         0.75
 1.         1.         1.         1.         1.         0.5
 1.         0.75       1.         1.         0.66666667 1.
 0.66666667 1.         1.         1.         1.         1.        ]
----------
Logistic Regression: 0.936111 +- (0.133824)
----------
Score TRAIN: 0.9545454545454546
Score TEST: 0.95
----------
Valor de Y_test en 9: 0
Predicción de Y_test: 0
Seguridad en predicción de Y_test: [1.00000000e+00 1.33889802e-19]
